![University of Tehran](./img/UT.png)
#   <font color='red'><center>AI CA 1<center></font> 
## <center>Dr. Fadaei<center>
### <center>Daniyal Maroufi<center>
### <center>810098039<center>

# Aim

This assignment aims to solve a problem using Informed and Uninformed search algorithms such as BTS, IDS, and A* algorithm. Finally, we will compare the time complexity of these algorithms.

# Problem Definition

## Intial State

The initial state is the state that our agent will start from. In this case, Gandalf starts from his state at a given position in the input.

## Action

The agent can do an action to reach another state. In this case, Gandolf can have six actions: going up, right, down, and left and picking a friend, and placing them in another position.

## Transition Model

UP action: If it is permitted, the agent will move from state (x,y) to state (x-1,y)

DOWN action: If it is permitted, the agent will move from state (x,y) to state (x+1,y)

RIGHT action: If it is permitted, the agent will move from state (x,y) to state (x,y+1)

LEFT action: If it is permitted, the agent will move from state (x,y) to state (a,y-1)

PICK action: the agent will pick the friend

PLACE action: the agent will place the friend

## Goal State

The goal state is that the agent has successfully done its tasks. In this case, Gandalf must place all the friends in their goal positions and reach the final point (Gandor).

# Algorithms

## BFS

The Breadth-First Search (BFS) is an algorithm for traversing or searching tree or graph data structures. It explores all the nodes at the present depth before moving to the next depth level. BFS is complete, and it will give us an optimal solution with time complexity of $O(b^d)$.

## IDS

The IDS or Iterative Deepening Search is an algorithm based on DFS which controls the depth before expanding another node. IDS leads to an optimal solution. Unlike DFS, its time complexity is $O(bm)$.

## A*

A* algorithm searches for the shortest path between the initial and the final state. It is used in various applications, such as maps.
In maps, the A* algorithm calculates the shortest distance between the source (initial state) and the destination (final state).
$g(n)$ is the cost of reaching the current state, and $h(n)$ is the heuristic function that estimates the cost of getting to the final state from the current node $n$.
A* search is complete, and its time complexity is $O(b^d)$.

# Classes

In [36]:
import queue
import time

## Node

In [48]:
class Node(object):
    def __init__(self,position):
        self.state=tuple(position)
        self.parent=None
        self.path=''
        self.picked_RF=tuple()
        self.placed_RFs=set()
    
    def getDist(self):
        return len(self.path)
    
    def setPlacedRFs(self,_placed_RFs):
        self.placed_RFs=_placed_RFs
    

## Problem

In [57]:
class Problem():
    def __init__(self, test_name):
        with open('./tests/'+test_name+'.txt') as f:
            self.n,self.m = tuple(map(int,f.readline().split()))
            self.gandalf_initial_pos = tuple(map(int,f.readline().split()))
            self.gandor = tuple(map(int,f.readline().split()))
            self.k,self.l = tuple(map(int,f.readline().split()))
            self.orks=[]
            self.RFs_initial_pos=[]
            self.RFs_goal_pos=[]
            for _ in range(self.k):
                x,y,c = tuple(map(int,f.readline().split()))
                self.orks.append((x,y,c))
            for _ in range(self.l):
                x,y = tuple(map(int,f.readline().split()))
                self.RFs_initial_pos.append((x,y))
            for _ in range(self.l):
                x,y = tuple(map(int,f.readline().split()))
                self.RFs_goal_pos.append((x,y))
            self.allRFs=dict(zip(self.RFs_initial_pos,self.RFs_goal_pos))
    
    def getInitialNode(self):
        node=Node(self.gandalf_initial_pos)
        return node
    
    def goalTest(self, node=Node((-1,-1))):
        if len(node.placed_RFs)==self.l and node.state==self.gandor:
            return True
        return False

    def isInRegionOfOrk(self,ork,node=Node((-1,-1))):
        if abs(ork[0]-node.state[0])+abs(ork[1]-node.state[1])<ork[2]:
            return True
        return False

    def countPresence(self,ork,node=Node((-1,-1))):
        presence=0
        while node.parent is None and self.isInRegionOfOrk(ork,node):
            presence+=1
            node=node.parent
        return presence

    def canGo(self,node=Node((-1,-1))):
        if node.state[0]<0 or node.state[0]>=self.n or node.state[1]<0 or node.state[1]>=self.m:
            return False
        for ork in self.orks:
            # count the number of RF's presence in the ork's region
            if self.countPresence(self,ork,node)>=ork[3]:
                return False
        return True
    
    def actions(self):
        return {'pick':'pick','place':'place','L':(-1,0),'R':(1,0),'U':(0,-1),'D':(0,1)}



In [54]:
test0=Problem('test_00')

In [55]:
test0.RFs_goal_pos


[(2, 30), (2, 31), (2, 32), (2, 33), (2, 34)]

In [56]:
def getHash(node=Node((-1,-1))):
    '''
    returns the hash of the state to save into a set
    hash(state+placed_RFs+picked_RF)
    '''
    return hash(str(node.state)+str(node.placed_RFs)+str(node.picked_RF))

In [61]:
def bfs(problem=Problem('test_00')):
    frontier=queue.Queue()
    frontierSet=set() # saves the hash of the states in the frontier
    explored=set() # saves the hash of the states that are explored
    seen_states, unique_seen_states = 0, 0
    root=problem.getInitialNode()
    frontier.put(root)
    frontierSet.add(getHash(root))
    if problem.goalTest(root):
        return 1,1,root.path
    while frontier:
        node=frontier.get()
        frontierSet.remove(getHash(node))
        explored.add(getHash(node))
        for action,transition in problem.actions().items():
            if action == 'pick':
                if node.picked_RF is None and node.state in problem.RFs_initial_pos and node.state not in node.placed_RFs:
                    child_with_RF=Node(node.state)
                    child_with_RF.parent=node
                    child.path=node.path
                    child_with_RF.picked_RF=problem.allRFs[node.state]
                    child_with_RF.placed_RFs=node.placed_RFs
                    seen_states+=1
                    frontier.put(child_with_RF)
                    frontierSet.add(getHash(child_with_RF))
            elif action == 'place':
                if node.state == node.picked_RF:
                    node.placed_RFs.add(node.state)
                    node.picked_RF=None
            else:
                child=Node((node.state[0]+transition[0],node.state[1]+transition[1]))
                child.parent=node
                child.path=node.path+action
                child.picked_RF=node.picked_RF
                child.setPlacedRFs(node.placed_RFs)
                seen_states+=1
                if getHash(child) not in explored and getHash(child) not in frontierSet:
                    unique_seen_states+=1
                    if problem.canGo(child):
                        if problem.goalTest(child):
                            return seen_states, unique_seen_states, child.path
                    frontier.put(child)
                    frontierSet.add(getHash(child))






In [62]:
bfs()

KeyboardInterrupt: 

In [30]:
dd.path='R'

In [31]:
dd.path+='L'

In [32]:
dd.path

'RL'

In [33]:
dd.getDist()

2